In [32]:
import math
from flashtext import KeywordProcessor
import datetime
import pandas as pd
from db_conn import get_connection
import alchemy_con
import spacy
import en_core_web_sm
# import nltk

start_time = datetime.datetime.now()
nlp = en_core_web_sm.load()
# chunk_parser = nltk.RegexpParser('''NP: {<DT>?<JJ>*<NN>}''')

In [2]:
engine = alchemy_con.get_engine()

In [3]:
hypers = pd.read_sql('select * from article_table_sentences_m where lower(sentence) like %(search_s)s', \
                     engine,\
                     params={'search_s':'%-hypertension%'})

hypers.head()

,id,sentence,table_id,drug,adverse_effect,frequency,table_title,foot_note
0,2172021,"Criterion , Randomized , STOP-Hypertension-2...",42838,None,None,None,Entry Criteria as Met by Trials Comparing Cal...,trial acronyms are defined in the references
1,2172027,"Criterion , STOP-Hypertension-2 (Ref . 4 ) :...",42838,None,None,None,Entry Criteria as Met by Trials Comparing Cal...,trial acronyms are defined in the references
2,2172033,"Criterion , Double-blinded or PROBE design P...",42838,None,None,None,Entry Criteria as Met by Trials Comparing Cal...,trial acronyms are defined in the references
3,2172039,"Criterion , Double-blinded , STOP-Hypertensi...",42838,None,None,None,Entry Criteria as Met by Trials Comparing Cal...,trial acronyms are defined in the references
4,2172044,"Criterion , Predefined clinical end points ;...",42838,None,None,None,Entry Criteria as Met by Trials Comparing Cal...,trial acronyms are defined in the references


In [4]:
keyword_processor = KeywordProcessor(case_sensitive=False)
words = ['hypertension'] #lower()
for d in words:
    keyword_processor.add_keyword(d.strip())
    keyword_processor.add_keyword(' '+d.strip()+' ')
    keyword_processor.add_keyword('-'+d.strip())
    keyword_processor.add_keyword(d.strip()+'-')
    keyword_processor.add_keyword('-'+d.strip()+'-')

True

True

True

True

True

In [36]:
update_list = []
for ind, row in hypers[:100].iterrows():
    ori_found = []
    sen = row['sentence']
    print(sen)
    found = keyword_processor.extract_keywords(sen.lower())
    if len(found)>0:
        _found = list(set(list(map(lambda x: x, found))))
        sen_strs = sen.strip().lower().split(' ')
#         doc = nlp(sen.strip().lower())
#         tree= chunk_parser.parse(nltk.pos_tag(nltk.word_tokenize(sen.strip().lower())))
#         for sub_tr in tree.subtrees():
#             print(' '.join([le[0] for le in sub_tr.leaves()]))
#         sen_strs = [c.text for c in doc.noun_chunks]
#         print(sen_strs)
        for f in _found:
            ori_found.extend(list(filter(lambda x: x.find(f)>=0, sen_strs)))
        print(ori_found)
        update_list.append({'id':id, 'adverse_event':' '+' , '.join(list(set(ori_found)))+' '})

 Criterion ,  Randomized , STOP-Hypertension-2 (Ref . 4 ) : +1 
['stop-hypertension-2']
 Criterion ,  STOP-Hypertension-2 (Ref . 4 ) : calcium channel blocker vs . diuretic or beta-blocker 
['stop-hypertension-2']
 Criterion ,  Double-blinded or PROBE design PROBE = Prospective , Randomized , Open , Blinded Endpoint evaluation design ; see test for details. , STOP-Hypertension-2 (Ref . 4 ) : +1 
['stop-hypertension-2']
 Criterion ,  Double-blinded , STOP-Hypertension-2 (Ref . 4 ) : PROBE 
['stop-hypertension-2']
 Criterion ,  Predefined clinical end points ; primary end points , STOP-Hypertension-2 (Ref . 4 ) : +1 
['stop-hypertension-2']
 Criterion ,  Total cardiovascular morbidity , STOP-Hypertension-2 (Ref . 4 ) : Stroke , myocardial infarction , all cardiovascular deaths 
['stop-hypertension-2']
 Criterion ,  Not short-acting calcium channel blocker ; specific drug used , STOP-Hypertension-2 (Ref . 4 ) : +1 
['stop-hypertension-2']
 Criterion ,  nifedipine gastro-intestinal tract s

In [20]:
doc.vocab()

TypeError: 'spacy.vocab.Vocab' object is not callable